# Configure Parameters for a Specific Test

This notebook guides model developers through using a simple classification model for bank customer churn dataset. It shows you how to set up the ValidMind Developer Framework and guide you through documenting a model using the ValidMind Developer framework. It shows how user can configure parameters for a test or set of tests in a specific section of document. For this simple demonstration, we use a bank customer churn dataset from Kaggle (https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data).

We will train a sample model and demonstrate the following documentation functionalities:

- Initializing the ValidMind Developer Framework
- Using a sample datasets provided by the library to train a simple classification model
- Configure a set of tests' parameters to generate document about the data and model

### ValidMind at a glance

We offer a platform for managing model risk, including risk associated with AI and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on doucumentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind:

<a href="https://docs.validmind.ai/guide/get-started.html">Get started</a> — The basics, including key concepts, and how our products work

<a href="https://docs.validmind.ai/guide/get-started-developer-framework.html">Get started with the ValidMind Developer Framework</a> — The path for developers, more code samples, and our developer reference




## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

### Install ValidMind Developer Framework


In [1]:
%pip install -q validmind


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Initializing the Python environment

In [2]:
import pandas as pd
import xgboost as xgb

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook. As you will see later, documentation projects are useful because they act as containers for model documentation and validation reports and they enable you to organize all of your documentation work in one place. 

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Customer Churn Model`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [3]:
## Replace the code below with the code snippet from your project ##


import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)
  

2024-01-19 11:40:02,916 - INFO(validmind.api_client): Connected to ValidMind. Project: [Demo] Customer Churn Model - Initial Validation (clr4sqr8z003muqy64hksfbnx)


### Preview the model's documentation template

All models are assigned a documentation template when registered. The template defines a list of sections that are used to document the model. Each section can contain any number of rich text and test driven blocks that populate the documentation. Test driven blocks are populated by running tests against the model.

We can preview the model documentation template for this project by running the following code:

In [4]:
vm.preview_template()

Accordion(children=(Accordion(children=(HTML(value='<p>Empty Section</p>'), Accordion(children=(HTML(value='<p…

## Load the demo dataset

For the purpose of this demonstration, we will use a sample dataset provided by the ValidMind library. 

In [5]:
# Import the sample dataset from the library
from validmind.datasets.classification import customer_churn as demo_dataset
# You can try a different dataset with:
# from validmind.datasets.classification import taiwan_credit as demo_dataset

df = demo_dataset.load_data()

#### Initialize a dataset object for ValidMind

Before running the test plan, we must first initialize a ValidMind dataset object using the `init_dataset` function from the `vm` module. This function takes in arguements: `dataset` which is the dataset that we want to analyze; `target_column` which is used to identify the target variable; `class_labels` which is used to identify the labels used for classification model training.

In [6]:
vm_dataset = vm.init_dataset(
    dataset=df,
    target_column=demo_dataset.target_column,
    class_labels=demo_dataset.class_labels
)

2024-01-19 11:40:08,559 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


### Documenting the model

We will need to preprocess the dataset and produce the training, test and validation splits first.

#### Prepocess the raw dataset

For demonstration purposes, we simplified the preprocessing using demo_dataset.preprocess which executes the following operations: 

In [7]:
train_df, validation_df, test_df = demo_dataset.preprocess(df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_val, y_val)],
    verbose=False,
)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=['error', 'logloss', 'auc'],
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

We can now initialize the training and test datasets into dataset objects using vm.init_dataset():

In [8]:
vm_train_ds = vm.init_dataset(
    dataset=train_df,
    target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df,
    target_column=demo_dataset.target_column
)

2024-01-19 11:40:08,807 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...
2024-01-19 11:40:08,809 - INFO(validmind.client): Pandas dataset detected. Initializing VM Dataset instance...


We also initialize a model object using vm.init_model():

In [9]:
vm_model = vm.init_model(
    model,
    train_ds=vm_train_ds,
    test_ds=vm_test_ds,
)

#### Run the template documentation suite

We are now ready to run the model's documentation tests as defined in its template. The following function runs every test in the template and sends all documentation artifacts to the ValidMind platform.

In [12]:
full_suite = vm.run_documentation_tests(
    inputs = {
        "dataset":vm_dataset,
        "model":vm_model
    }
)

#### Configuration of parameters for model diagnosis tests 

Each test has its default parameters and their values depending on the use case you are trying to solve. ValidMind's developer framework exposes these parameters at the user level so that they can be adjusted based on requirements.

The config can be applied to a specific test to override the default configuration parameters.

The format of the config is:
```
config = {
    "<test1_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
     "<test2_id>": {
        "<default_param_1>": value,
        "<default_param_2>": value,
    },
}
```

Users can input the configuration to `run_documentation_tests()` and `run_test_suite()` using **`config`**, allowing fine-tuning the suite according to the specific configuration requirements. 

In [13]:
config = {
    "validmind.model_validation.sklearn.OverfitDiagnosis": {
        "cut_off_percentage": 3,
        "feature_columns": ["Age", "Balance", "Tenure", "NumOfProducts"]
    },
    "validmind.model_validation.sklearn.WeakspotsDiagnosis": {
        "features_columns": ["Age", "Balance"],
        "accuracy_gap_threshold": 85,
    },
    "validmind.model_validation.sklearn.RobustnessDiagnosis": {
        "features_columns": ["Balance", "Tenure"],
        "scaling_factor_std_dev_list": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        "accuracy_decay_threshold": 4,
    }
}

full_suite = vm.run_documentation_tests(
    inputs = {
        "dataset":vm_dataset,
        "model":vm_model
    },
    section="model_diagnosis",
    config=config,
)

### Next steps
You can look at the results of this test plan right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI:

Log back into the Platform UI

Go to **Documentation Projects > YOUR_DOCUMENTATION_PROJECT > Documentation.**

Expand ** Model Development**

What you can see now is a more easily consumable version of the model diagnosis tests you just performed, along with other parts of your documentation project that still need to be completed.

If you want to learn more about where you are in the model documentation process, take a look at <a href="https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework"> How do I use the framework? </a>.